# Preprocessing Survey Result

In [1]:
import pandas as pd
import numpy as np
from datetime import time

# Load the survey data
survey_data = pd.read_excel('stress_dataset/SurveyResults.xlsx', sheet_name=None)
all_surveys = pd.concat(survey_data.values(), ignore_index=True)
all_surveys.columns = (
    all_surveys.columns
    .str.strip()
    .str.lower()
    .str.replace(r"[()\-\'\"]", "", regex=True)  # Drop ()-'" (patient's => patients)
    .str.replace(r"[\s\,]", "_", regex=True)  # Space and comma to underscore (admin lab, pharma => admin_lab_pharma)
    .str.replace(r"_+", "_", regex=True)  # Reduce multiple underscores to a single underscore (admin___lab => admin_lab)
)
display(all_surveys.head())
# display(all_surveys)

,id,start_time,end_time,duration,date,stress_level,covid_related,treating_a_covid_patient,patient_in_crisis,patient_or_patients_family,doctors_or_colleagues,administration_lab_pharmacy_radiology_or_other_ancilliary_services,increased_workload,technology_related_stress,lack_of_supplies,documentation,competency_related_stress,saftey_physical_or_physiological_threats,work_environment_physical_or_others:_work_processes_or_procedures,description
0,5C,08:00:00,09:00:00,01:00:00,2020-04-15,1,0,1,0,1,0,0,0,0,0,0,0,0,0,na
1,5C,17:31:00,17:58:00,00:27:00,2020-04-14,1,0,1,0,1,0,0,1,0,0,0,0,0,0,na
2,E4,15:32:00,15:37:00,00:05:00,2020-04-18,2,0,1,0,1,0,0,0,0,0,0,0,0,0,Spoke with family regarding patient's decline ...
3,E4,14:05:00,14:11:00,00:06:00,2020-04-18,2,0,0,0,1,0,0,0,0,0,0,0,0,0,Was placing another FaceTime call to a patient...
4,7A,13:52:00,14:03:00,00:11:00,2020-04-18,2,0,1,0,0,0,0,1,0,0,0,0,0,1,na


In [2]:
# Convert Date and Time columns into timestamps
import pytz

timezone = pytz.timezone('America/Chicago')

all_surveys['start_timefull'] = pd.to_datetime(all_surveys['date'].astype(str) + " " + all_surveys['start_time'].astype(str)).dt.tz_localize(timezone, ambiguous='NaT', nonexistent='NaT')
all_surveys['end_timefull'] = pd.to_datetime(all_surveys['date'].astype(str) + " " + all_surveys['end_time'].astype(str)).dt.tz_localize(timezone, ambiguous='NaT', nonexistent='NaT')

all_surveys['start_timestamp'] = all_surveys['start_timefull'].apply(lambda x: x.timestamp()).astype(int)
all_surveys['end_timestamp'] = all_surveys['end_timefull'].apply(lambda x: x.timestamp()).astype(int)

### Check inconsistence duration

In [3]:
def time_to_seconds(time_val):
    if isinstance(time_val, str):  # If it's a string, process normally
        try:
            hours, minutes, seconds = map(int, time_val.split(':'))
            return hours * 3600 + minutes * 60 + seconds
        except (ValueError, AttributeError):
            return np.nan
    elif isinstance(time_val, time):  # If it's a datetime.time object, convert directly
        return time_val.hour * 3600 + time_val.minute * 60 + time_val.second
    else:
        return np.nan  # If it's neither, return NaN

all_surveys['duration_seconds'] = all_surveys['duration'].apply(time_to_seconds)
all_surveys['calculated_duration'] = (all_surveys['end_timefull'] - all_surveys['start_timefull']).dt.total_seconds()
all_surveys['duration_mismatch'] = abs(all_surveys['duration_seconds'] - all_surveys['calculated_duration']) > 0
# Check for any duration mismatches then we can warn about them
mismatch_rows = all_surveys[all_surveys['duration_mismatch']]
if not mismatch_rows.empty:
    print("⚠️ Warning: Duration mismatches detected at the following row indices:")
    print(mismatch_rows.index.tolist())
    display(mismatch_rows)

# Drop the unnecessary columns
all_surveys.drop(columns=['calculated_duration', 'duration_mismatch'], inplace=True)
# all_surveys.drop(columns=['calculated_duration', 'duration_mismatch'], inplace=True)

# Sort the surveys by start time and ID
all_surveys = all_surveys.sort_values(by=['start_timestamp', 'id'], ascending=[True, True])

⚠️ Warning: Duration mismatches detected at the following row indices:
[333]


,id,start_time,end_time,duration,date,stress_level,covid_related,treating_a_covid_patient,patient_in_crisis,patient_or_patients_family,...,saftey_physical_or_physiological_threats,work_environment_physical_or_others:_work_processes_or_procedures,description,start_timefull,end_timefull,start_timestamp,end_timestamp,duration_seconds,calculated_duration,duration_mismatch
333,BG,23:53:00,00:05:00,00:08:00,2020-12-02,na,na,na,na,na,...,na,na,na,2020-12-02 23:53:00-06:00,2020-12-02 00:05:00-06:00,1606974780,1606889100,480,-85680.0,True


### Data amputation

In [4]:
# Convert "na" string to NaN

all_surveys.replace("na", np.nan, inplace=True)

/tmp/ipykernel_3073210/1394396253.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_surveys.replace("na", np.nan, inplace=True)


In [5]:
# Current stategy: Drop entire row NA values, but we keep some statistics about the NA values
exclude_columns = ['id', 'date', 'start_time', 'end_time', 'duration', 'duration_seconds', 'start_timefull', 'end_timefull', 'start_timestamp', 'end_timestamp']
columns_to_check = [col for col in all_surveys.columns if col not in exclude_columns]

# Identify full na row
all_surveys['is_na_entry'] = all_surveys[columns_to_check].isna().all(axis=1)
all_surveys['date_datetime'] = all_surveys['start_timefull'].dt.date

all_surveys['na_entry_before_count'] = 0
all_surveys['na_entry_before_duration'] = 0
all_surveys['na_entry_after_count'] = 0
all_surveys['na_entry_after_duration'] = 0

for (date, nurse_id), group in all_surveys.groupby(['date_datetime', 'id']):
    na_entries = []
    valid_indices = []

    for index, row in group.iterrows():
        if row['is_na_entry']:
            na_entries.append((index, row['duration_seconds']))  # Store NA row index and duration
        else:
            # Assign NA entry counts & duration before
            all_surveys.at[index, 'na_entry_before_count'] = len(na_entries)
            all_surveys.at[index, 'na_entry_before_duration'] = sum(d for _, d in na_entries)

            valid_indices.append(index)
            na_entries = []

    if valid_indices:
        last_valid_index = valid_indices[-1]
        all_surveys.at[last_valid_index, 'na_entry_after_count'] = len(na_entries)
        all_surveys.at[last_valid_index, 'na_entry_after_duration'] = sum(d for _, d in na_entries)

    for prev_index, index in zip(valid_indices, valid_indices[1:]):
        all_surveys.at[prev_index, 'na_entry_after_count'] = all_surveys.at[index, 'na_entry_before_count']
        all_surveys.at[prev_index, 'na_entry_after_duration'] = all_surveys.at[index, 'na_entry_before_duration']

# Clean up the temporary columns
all_surveys.drop(columns=['is_na_entry', 'date_datetime'], inplace=True)

# Drop rows where all selected columns are 'na'
all_surveys = all_surveys.dropna(subset=columns_to_check, how='all')

# Check if any remaining NaN values exist and print their positions
columns_to_check.remove('description')
na_cells = all_surveys[columns_to_check].isna().any(axis=1)
if na_cells.any():
    print("⚠️ Warning: Some cells still contain NaN values after dropping full NaN rows.")
    display(all_surveys[na_cells])
else:
    print("✅ No NaN values remaining in the dataset.")

display(all_surveys.head())

✅ No NaN values remaining in the dataset.


,id,start_time,end_time,duration,date,stress_level,covid_related,treating_a_covid_patient,patient_in_crisis,patient_or_patients_family,...,description,start_timefull,end_timefull,start_timestamp,end_timestamp,duration_seconds,na_entry_before_count,na_entry_before_duration,na_entry_after_count,na_entry_after_duration
1,5C,17:31:00,17:58:00,00:27:00,2020-04-14,1.0,0.0,1.0,0.0,1.0,...,NaN,2020-04-14 17:31:00-05:00,2020-04-14 17:58:00-05:00,1586903460,1586905080,1620,0,0,0,0
0,5C,08:00:00,09:00:00,01:00:00,2020-04-15,1.0,0.0,1.0,0.0,1.0,...,NaN,2020-04-15 08:00:00-05:00,2020-04-15 09:00:00-05:00,1586955600,1586959200,3600,0,0,0,0
7,E4,08:15:00,08:24:00,00:09:00,2020-04-18,2.0,0.0,0.0,0.0,0.0,...,Was trying to get organized after being pulled...,2020-04-18 08:15:00-05:00,2020-04-18 08:24:00-05:00,1587215700,1587216240,540,0,0,0,0
8,E4,09:32:00,09:38:00,00:06:00,2020-04-18,2.0,0.0,1.0,0.0,1.0,...,Was assessing a COVID patient who required a l...,2020-04-18 09:32:00-05:00,2020-04-18 09:38:00-05:00,1587220320,1587220680,360,0,0,0,0
4,7A,13:52:00,14:03:00,00:11:00,2020-04-18,2.0,0.0,1.0,0.0,0.0,...,NaN,2020-04-18 13:52:00-05:00,2020-04-18 14:03:00-05:00,1587235920,1587236580,660,0,0,0,0


### Drop unncessaries columns and duplicates data

In [6]:
# If we don't need original date time anymore
all_surveys.drop(columns=['duration', 'date', 'start_time', 'end_time', 'start_timefull', 'end_timefull'], inplace=True)

# Remove duplicate rows, but won't happen much since we don't have duplicates
# all_surveys.drop_duplicates(inplace=True)

### Save

In [7]:
# Save the cleaned dataset to CSV
all_surveys.to_csv('Preprocessed_SurveyResults.csv', index=False)

# Load and view the preprocessed survey data from CSV
preprocessed_survey_data = pd.read_csv('Preprocessed_SurveyResults.csv')
display(preprocessed_survey_data.head())

,id,stress_level,covid_related,treating_a_covid_patient,patient_in_crisis,patient_or_patients_family,doctors_or_colleagues,administration_lab_pharmacy_radiology_or_other_ancilliary_services,increased_workload,technology_related_stress,...,saftey_physical_or_physiological_threats,work_environment_physical_or_others:_work_processes_or_procedures,description,start_timestamp,end_timestamp,duration_seconds,na_entry_before_count,na_entry_before_duration,na_entry_after_count,na_entry_after_duration
0,5C,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,NaN,1586903460,1586905080,1620,0,0,0,0
1,5C,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,1586955600,1586959200,3600,0,0,0,0
2,E4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,Was trying to get organized after being pulled...,1587215700,1587216240,540,0,0,0,0
3,E4,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,Was assessing a COVID patient who required a l...,1587220320,1587220680,360,0,0,0,0
4,7A,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,NaN,1587235920,1587236580,660,0,0,0,0
